In [2]:
%autoreload 2

In [3]:
from collections import defaultdict
from argparse import Namespace
import copy
import gzip
import itertools
import os
import pickle
import sys
import typing

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tatsu
import tatsu.ast
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import tqdm
import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.pipeline import Pipeline

sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
from src import fitness_energy_utils as utils
from src.fitness_energy_utils import NON_FEATURE_COLUMNS
from src.fitness_features import *
from src.ast_counter_sampler import *
from src.ast_mcmc_regrowth import *

In [4]:
fitness_df = utils.load_fitness_data()
print(fitness_df.src_file.unique())
fitness_df.head()

['interactive-beta.pddl' 'ast-real-regrowth-samples.pddl']


,Index,src_file,game_name,domain_name,real,original_game_name,all_variables_defined,all_variables_used,all_preferences_used,setup_objects_used,...,ast_ngram_n_5_0,ast_ngram_n_5_1,ast_ngram_n_5_2,ast_ngram_n_5_3,ast_ngram_n_5_4,ast_ngram_n_5_5,ast_ngram_n_5_6,ast_ngram_n_5_7,ast_ngram_n_5_8,ast_ngram_n_5_9
0,0,interactive-beta.pddl,6172feb1665491d1efbce164-0,medium-objects-room-v1,1,6172feb1665491d1efbce164-0,1,1,1,1.00,...,1,1,1,1,1,1,1,1,1,1
1,1,interactive-beta.pddl,5f77754ba932fb2c4ba181d8-2,many-objects-room-v1,1,5f77754ba932fb2c4ba181d8-2,1,1,1,1.00,...,1,1,0,1,0,1,0,1,1,1
2,2,interactive-beta.pddl,614b603d4da88384282967a7-3,many-objects-room-v1,1,614b603d4da88384282967a7-3,1,1,1,0.00,...,1,0,0,0,0,0,0,0,0,0
3,3,interactive-beta.pddl,5bc79f652885710001a0e82a-5,few-objects-room-v1,1,5bc79f652885710001a0e82a-5,1,1,1,0.00,...,1,1,1,1,1,1,1,1,1,1
4,4,interactive-beta.pddl,614dec67f6eb129c3a77defd-6,medium-objects-room-v1,1,614dec67f6eb129c3a77defd-6,1,1,1,0.25,...,1,1,0,1,1,1,1,1,1,1


# Plan of attack
* Use the code in `ast_counter_sampler.py` to generate a sample from the MLE
* Score it with a fitness function adapted to working with a single example at a time
* At each subsequent iteration, regrow the game from a random node, and score the regrowth
* Since we have energy scores before and after, we can either accept greedily if $E_{new} < E_{old}$, or accept with probability $\exp(\beta (E_{old} - E_{new} ))$

In [5]:
DEFAULT_ARGS = argparse.Namespace(
    grammar_file=os.path.join('..', DEFAULT_GRAMMAR_FILE),
    parse_counter=False,
    counter_output_path=os.path.join('..', DEFAULT_COUNTER_OUTPUT_PATH),
    random_seed=DEFUALT_RANDOM_SEED,
)

In [6]:
FITNESS_MODEL_PATH = '../models/cv_fitness_model_2023_02_02.pkl.gz'
FITNESS_FEATURIZER_PATH = '../models/fitness_featurizer_2023_02_02.pkl.gz'

feature_names = [c for c in fitness_df.columns if c not in NON_FEATURE_COLUMNS]

mcmc = MCMCRegrowthSampler(DEFAULT_ARGS, feature_names,
    FITNESS_MODEL_PATH, FITNESS_FEATURIZER_PATH, greedy_acceptance=True, 
    plateau_patience_steps=100, max_steps=5000)  #   acceptance_temperature=10.0, 

In [7]:
# %prun -r 
mcmc.multiple_samples(10, verbose=1, should_tqdm=True)
print([x[2] for x in mcmc.samples])

  0%|          | 0/10 [00:00<?, ?it/s]

Plateaued at step 991 with energy 6.0424604415893555
Plateaued at step 567 with energy 2.599466323852539
Plateaued at step 279 with energy 6.443458080291748
Plateaued at step 458 with energy 3.8171777725219727
Plateaued at step 538 with energy 0.5992338061332703
Plateaued at step 573 with energy 5.847500801086426
Plateaued at step 864 with energy 7.560303211212158
Plateaued at step 133 with energy 7.8907976150512695
Plateaued at step 1127 with energy 5.971003532409668
Plateaued at step 900 with energy 2.6397881507873535
[6.0424604415893555, 2.599466323852539, 6.443458080291748, 3.8171777725219727, 0.5992338061332703, 5.847500801086426, 7.560303211212158, 7.8907976150512695, 5.971003532409668, 2.6397881507873535]


In [1]:
mcmc.visualize_sample(4)

NameError: name 'mcmc' is not defined

### Notes from looking at these games
* Revisit the logic in `pref_forall_correct_types` and `pref_forall_correct_arity` -- it might want to be a 1/0/-1 instead of a 1/0
* Index 4 somehow caught onto the fact that `(agent_holds color)` is not penalized heavily, but also the same `pref_forall_...` stuff
* Features for different counts of items under a then, and/or for any two consecutive children of a then with the same modal?
* This sampler generates oddities that are rare enough in the training set to not get a significant weight -- see the terminal and scoring sections of indices 3 and 8, for examples 

In [2]:
print(ast_printer.ast_to_string(samples[8][0], '\n'))

NameError: name 'ast_printer' is not defined

In [ ]:
fitness_featurizer = build_fitness_featurizer(DEFAULT_ARGS)
features = fitness_featurizer.parse(samples[7][0], 'mcmc', True)
{k: v for k, v in features.items() if 'correctly' in k or 'forall' in k or 'no' in k}

In [ ]:
with gzip.open(DEFAULT_FITNESS_FUNCTION_PATH, 'rb') as f:
    cv_fitness_model = pickle.load(f)

In [ ]:
weights = cv_fitness_model.named_steps['fitness'].model.fc1.weight.data.detach().numpy().squeeze()
bias = cv_fitness_model.named_steps['fitness'].model.fc1.bias.data.detach().numpy().squeeze()
print(weights.mean(), bias)

In [ ]:
plt.hist(weights, bins=100)
plt.title('Fitness Model Weights')
plt.xlabel('Weight magnitude')
plt.ylabel('Count')

In [12]:
import ast_crossover_sampler
asts = [ast for ast in cached_load_and_parse_games_from_file('../dsl/interactive-beta.pddl', 
    mcmc.grammar_parser, False, relative_path='..')]

mcmc_with_crossover = MCMCRegrowthCrossoverSampler(DEFAULT_ARGS, 
    ast_crossover_sampler.CrossoverType.SAME_RULE, asts, 0.25,
    '../models/cv_fitness_model_2022_12_26.pkl.gz', 
    greedy_acceptance=True, plateau_patience_steps=20, acceptance_temperature=10.0, max_steps=1000)

In [13]:
mcmc_with_crossover.multiple_samples(10, verbose=2, should_tqdm=True)
print([x[2] for x in mcmc_with_crossover.samples])

  0%|          | 0/10 [00:00<?, ?it/s]

Sampling exception, skipping sample
Regrowth generated identical games, repeating
Accepted step 1 with fitness 7.857532501220703
Accepted step 2 with fitness 7.784302711486816
Accepted step 3 with fitness 7.775148391723633
Accepted step 4 with fitness 7.692765235900879
Accepted step 6 with fitness 7.536055564880371
Regrowth generated identical games, repeating
Accepted step 8 with fitness 7.5235395431518555
Accepted step 10 with fitness 7.4317522048950195
Accepted step 11 with fitness 6.490839004516602
Accepted step 17 with fitness 5.810407638549805
Accepted step 19 with fitness 5.400604248046875
Accepted step 28 with fitness 5.354710578918457
Accepted step 44 with fitness 4.757689476013184
Accepted step 49 with fitness 4.688848495483398
Accepted step 68 with fitness 4.574114799499512
Accepted step 75 with fitness 4.34464693069458
Regrowth generated identical games, repeating
Accepted step 93 with fitness 3.9308552742004395
Regrowth generated identical games, repeating
Accepted step 10

In [14]:
print([x[2] for x in mcmc_with_crossover.samples])

[-0.5954062938690186, -0.4696420133113861, 1.7651313543319702, 5.2624006271362305, 0.8746525049209595, 1.2288413047790527, 0.9395236968994141, 1.8448952436447144, 0.8919482231140137, 1.2082358598709106]
